# Data munging with dplyr

### Stephen Elston
### Data Science 350

## Introduction

Data rarely arrives in the form we need for analysis. In fact, it is has been estimated that data scientists spend 80% of their time preparing data (some say [the other 20% is spent complaining about preparing data:)](https://twitter.com/bigdataborat/status/306596352991830016?lang=en). This process of integrating and filtering data is commonly know as data munging. 

This notebook contains exercises in using dplyr for data munging. The R dplyr package contains a widely used, powerful, flexible and efficient set of functions for data munging. Further, dplyr uses a regular gramar for defining complex data munging pipelines. 

## Why dplyr?

As already stated dplyr is a powerful, flexible and efficient R package, specifically designed for data munging. The dplyr package is a subset of the Tidyverse group of data munging and utilitity packages https://cran.r-project.org/web/packages/tidyverse/index.html  



Excellent documentation is available for the dplyr package:
- An introduciton to dplyr can be found at http://rstudio-pubs-static.s3.amazonaws.com/11068_8bc42d6df61341b2bed45e9a9a3bf9f4.html 
- The dplyr cheat sheet is an excellent summary and quick reference for dplyr and some other Tidyverse packages: https://www.rstudio.com/wp-content/uploads/2015/02/data-wrangling-cheatsheet.pdf 
- A number of special topics are covered in the several vignettes you can find in CRAN: https://cran.r-project.org/web/packages/dplyr/index.html

An in-depth treatment of the entire Tidyverse can be found in the [book by Wickham and  Grolemund](http://shop.oreilly.com/product/0636920034407.do).

## Load the Data Set

Use the code in the cell below to load the data set. **Make sure you have the .csv file in your working directory!**

In [ ]:
read.auto = function(file = 'Automobile price data _Raw_.csv'){
  ## Read the csv file
  auto.price <- read.csv(file, header = TRUE, 
                      stringsAsFactors = FALSE)

  ## Coerce some character columns to numeric
  numcols <- c('price', 'bore', 'stroke', 'horsepower', 'peak.rpm')
  auto.price[, numcols] <- lapply(auto.price[, numcols], as.numeric)

  ## Remove cases or rows with missing values. In this case we keep the 
  ## rows which do not have nas. 
  auto.price[complete.cases(auto.price), ]
}
auto.price = read.auto()

## Verbs in dplyr

The dplyr package contains a large number of operators, called verbs, which you can apply to a data frame. The name of each verb is indicative of the action the operator will take. For example, you can apply a boolean filter to the rows of a data fram with the `filter` verb. 

Each dplyr verb uses a regular gramar or syntax. This means that you don't have to remember a lot of syntactic detail to use dplyr. The basic gramar of each dplyr verb is:

$$df.result = verb\_name(df, arguments)$$

where `df` is the original data frame, and `arguments` are the one or more arguments to the operator. Thats all there is to it! 

In the rest of this section you will apply some of the most commonly used dplyr verbs to the data set. 

### Filter the data set

Here is an example to make this a bit more concrete. The code below applies a dpyr filter to the auto pricing data. The result will only be the cases or rows of autos made by the Audi company. 

Notice that you can just type the name of the column(s) you need for your boolean filter. You do not need any quotes, or the usual R `df$make` or df[, 'make'] notation. This syntax saves you a lot of typing when you used dplyr compared to the conventional R notation. Additionally, you gain speed and scalability from the more efficient dplyr implementation of the verbs. 

Execute the code in the cell below and note the result. 

In [ ]:
require(dplyr)
df = filter(auto.price, make == 'audi')
df

The printed output shows the 6 cases of autos made by Audi in the data set. Autos made by all other manufacturers have been filtered out.

**Your turn!** Create and execute the code in the cell below to filter for autos with 4wd type drive wheels.

### Take slice from data frame

You have just selected cases from a data set based  on a logical relationship. You can also select rows, or cases, using the row number with the  `slice` function. Run the code in the cell below to select a subset of rows.

In [ ]:
df.slice = slice(auto.price, 20:30)
df.slice

The 11 row slice has been sub-selected from the data frame. 

### Random sample a data frame

Data scientists often need randomly chosen subsets of data. Using the dplyr `sample_frac` and `sample_n` functions allow you randomly sample a fraction or a number of rows respectively. Run the code in the cell below to take a random sample of 1/2 of the rows in the `df.slice` data frame.

In [ ]:
df.rand = sample_frac(df.slice, 0.5)
df.rand

Examine the result, with 6 (approximately 1/2 of 11) of 11 original rows.

***
**Note:** if you want specific number of random samples you can use the `sample_n` verb. 
***

### Select columns

Now, say that we really only want a few attributes, or columns, from the Audi car data. Run the code in the cell below to `select` the columns listed. 

In [ ]:
df2 = select(df, drive.wheels, wheel.base, curb.weight, horsepower, price)
df2

The columns listed have been selected from the full Audi data set.

What if you only wish to remove a few columns from the data set? You do not need to list all the columns you want, which could be a large number. Instead, just prefix the names of the columns you wish to eliminate with the '-' operator. 

**Your Turn:** Use the dplyr `select` verb to remove three columns (`symboling`, `normalized.losses`, `engine.size`) from the data frame `df`. Examine and verify the result.

Examine the data frame printed above. Notice that the columns listed with the `-` operator have been eliminated.

### Arrange rows in a data frame

You have learned how to select the rows and columns from a data frame. But what if you want to reorder the rows in a data frame. You can use the dplyr `arrange` verb to reorder the rows. Execute the code in the cell below to sort the data frame first by drive wheels and then by auto price, and examine the result.

In [ ]:
df2 = arrange(df2, drive.wheels, price)
df2

### Compute a new column

Often you will need a column computed using the values of one or more columns in the original data set. The dplyr `mutate` verb allows you to efficiently add a new computed column to your data frame. Run the example below to add a colummn with the auto weight in killograms, rather than pounds, and the weight in pounds per unit of hoursepower.

In [ ]:
df4 = mutate(df2, curb.weight.kg = curb.weight / 2.205, weight.horsepower = curb.weight / horsepower)
select(df4, curb.weight, curb.weight.kg, weight.horsepower)

In the results printed above you can see the the curb weight of each Audi car in pounds and killograms, and the weight per unit of horsepower.

If you don't want to retain the orriginal columns from your calculation you can use the `transmute` verb. If you which to apply the same function to every column use the `:mutate_each` verb, which is similar to the standard R `lapply`. 

### Compute summaries of a data frame

Very often in data analysis you will need to compute summuary statstics for your data set. The dplyr `summary` function is designed just of this purpose. Run the code in the cell below to compute some summary statistics of a data frame.

In [ ]:
summarise(df4, mean.curb.weight = mean(curb.weight, na.rm = TRUE), sd.curb.weight = sd(curb.weight, na.rm = TRUE), 
         max.curb.weight = max(curb.weight), min.curb.weight = min(curb.weight))

The summary statistics for the Audi cars are computed and displayed. 

### Count cases

Another useful way to summarize data is by counts. The dplyr `count` verb lets you do just this. Run the code in the cell below to compute the counts of autos by body style and number of cylinders.

In [ ]:
count(auto.price, body.style, num.of.cylinders)

The summary of the original data frame by counts shows that there are a few popular configurations, like four cylinder hatachbacks, and many one or few of a kind configurations. 

**Your Turn:**  Now that you have a bit of experience with some of the most common dplyr verbs, try the following. Compute the weights of the autos made by Dodge in killograms, and the horse power of the auto per killogram of weight. Hint, you will need to use the `mutate` verb twice.  

## Chaining Verbs

You have expored some of the most commonly used dplyr verbs. By now, you should have a feel for the syntax used. Next, you will learn how to chain verbs to create complex data munging operations. 

The syntax of dplyr supports a chaining operator, `%>%`. With the chaining operator you chain together a data frame and any number of verbs to create a complex data mungig operation. The basic gramar is really simple:

$$result = df\ \%>\%\ verb1(arguments)\ \%>\%\ verb2(arguments)\ \%>\% ...$$

Where `df` is your data frame. Notice that you only need to specify `df` at the start of the chain. You only need to specify the arguments of the verbs in the chain, as the data frame name is implicit. Additionally, the operations are applied to each intermediate result without creating data frame copies. This execution model makes the verb chain fast and efficient, and reduces the chance of running out of memory, a frequent source of frustration with R. 

### A first verb chain

You will now apply a dplyr verb chain to compute the same result you achieved before with two distinct steps. The verb chain shown below selects autos made by Audi with the `filter` verb and then selcts 5 columns from that result. Notice the simple gramar of the verb chain. 

Run this code and examine the result. 

In [ ]:
df2 = auto.price %>% 
        filter(make == 'audi') %>% 
        select(drive.wheels, wheel.base, curb.weight, engine.size, price)
df2

Compare this result to the one you saw before. The result is exactly the same as you achieved with applying two verbs in two seperate steps. 

The above chain only uses two verbs. However, a verb chain can use any number of verbs you need. 

**Your Turn:** In a previous exercise you compute the weights of the autos made by Dodge in killograms, and the horse power of the autos per killogram of weight. Preform this same set of operations using a verb chain. Compare the results. Is there any difference?

### Use group_by in a verb chain

Next you will use a verb chain which includes the dplyr `group_by` verb. While you can use `group_by` in a simple line of code, the power of this operator really shines when used in a verb chain. 

For example, say you want to compare some summary statistics of auto prices organized by type of drive wheels. The code in the cell below does just this. The dplyr count function `n()` is used to compute the number of cases in each group. Execute this code and examine the result.

In [ ]:
auto.price %>% group_by(drive.wheels) %>% 
    summarise(count = n(), mean.price = mean(price, na.rm = TRUE), sd.price = sd(price, na.rm = TRUE), 
         max.price = max(price), min.price = min(price))

**Your Turn** Here is one final challange for you. Create a dplyr verb chain to compute the count, mean weight, standard deviation, maximum weight, minimum weight for toyota autos grouped by body style.  

#### Copyright 2017, Stephen F Elston. All rights reserved.